In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob, os
import datetime
%matplotlib inline

# Reading and consolidating excel files

In [2]:
# files_path = '../data/IMU Dataset'
# files = glob.glob(files_path)
# folders = ['ADLs', 'Falls', 'Near_Falls']

# df = pd.DataFrame()

# for i in range(1,11):
#     print('Subject: '+str(i))
#     for folder in folders:
#         print('-- Trial Type: '+str(folder))
#         path = files_path+'/sub'+str(i)+'/'+str(folder)+'/*.xlsx'
#         files = glob.glob(path)

#         # create a list of dataframes, one for each file, and assign a column with the name of the file
#         dfs_list = [pd.read_excel(fp).assign(FileName = os.path.basename(fp)) for fp in files]

#         df_temp = pd.DataFrame()
#         df_temp = pd.concat(dfs_list)
#         df_temp['subject'] = i
#         df_temp['trial_type'] = folder
        
#         df = df.append(df_temp)

In [3]:
# save the dataframe in a Pickle file
# pickle.dump(df, open("dataset_raw.p", "wb"))

# load the dataframe
df = pickle.load(open("../data/dataset_raw.p", "rb"))

In [4]:
import warnings
warnings.filterwarnings("ignore")

## Including implicit features

In [5]:
df['trial_type'].value_counts()

ADLs          498810
Falls         403402
Near_Falls    288157
Name: trial_type, dtype: int64

In [6]:
# 'Trial Type' is not ordinal, so we will need dummy variables

# extraction of the trial number from the file name
# Not Ordinal: necessary dummy variables
df['trial_num_original'] = df['FileName'].apply(lambda x: x.replace('.xlsx','').replace('trial','').split('_')[-1]).astype(int)

# extraction of the trial subtype (slip, trip, pick object from ground, etc) from the file name
# Necessary dummy variables
df['trial_subtype'] = df['FileName'].apply(lambda x: x.split('_')[1])

# column with the real date and time of the measurement
# according to the README.txt, the Time column is the number of microseconds from 01/01/1970
initial = datetime.datetime(1970,1,1,0,0,0)
df['time_datetime'] = df.Time.apply(lambda x: (initial + datetime.timedelta(microseconds=x)))
# creating the column 'Time_datetime' in a pandas datetime format
df['time_datetime'] = df.time_datetime.apply(lambda t: pd.datetime(t.year,t.month,t.day,t.hour,t.minute,t.second,t.microsecond))

###### The first 3 characters of the file name seems to be a supercategory of the trial_subtype.
###### This way, I believe we can discard it and use just trial_subtype
df['target'] = np.where(df['trial_type'] == 'Falls', 1, 0)
df.head(2)

,Time,r.ankle Acceleration X (m/s^2),r.ankle Acceleration Y (m/s^2),r.ankle Acceleration Z (m/s^2),r.ankle Angular Velocity X (rad/s),r.ankle Angular Velocity Y (rad/s),r.ankle Angular Velocity Z (rad/s),r.ankle Magnetic Field X (uT),r.ankle Magnetic Field Y (uT),r.ankle Magnetic Field Z (uT),...,waist Magnetic Field X (uT),waist Magnetic Field Y (uT),waist Magnetic Field Z (uT),FileName,subject,trial_type,trial_num_original,trial_subtype,time_datetime,target
0,1304799112429680,-9.691464,-0.240769,-1.514349,0.218779,0.055136,-0.011516,33.449158,19.266598,15.818048,...,45.724129,-24.729599,-24.925210,JXL_SQ_trial1.xlsx,1,ADLs,1,SQ,2011-05-07 20:11:52.429680,0
1,1304799112437500,-9.698196,-0.245077,-1.509888,0.213975,0.059683,-0.016308,33.449158,19.266598,15.818048,...,43.054999,-27.175334,-17.818824,JXL_SQ_trial1.xlsx,1,ADLs,1,SQ,2011-05-07 20:11:52.437500,0


## Create column with counting of time starting from 0 for each trial

In [7]:
df.head(2)

,Time,r.ankle Acceleration X (m/s^2),r.ankle Acceleration Y (m/s^2),r.ankle Acceleration Z (m/s^2),r.ankle Angular Velocity X (rad/s),r.ankle Angular Velocity Y (rad/s),r.ankle Angular Velocity Z (rad/s),r.ankle Magnetic Field X (uT),r.ankle Magnetic Field Y (uT),r.ankle Magnetic Field Z (uT),...,waist Magnetic Field X (uT),waist Magnetic Field Y (uT),waist Magnetic Field Z (uT),FileName,subject,trial_type,trial_num_original,trial_subtype,time_datetime,target
0,1304799112429680,-9.691464,-0.240769,-1.514349,0.218779,0.055136,-0.011516,33.449158,19.266598,15.818048,...,45.724129,-24.729599,-24.925210,JXL_SQ_trial1.xlsx,1,ADLs,1,SQ,2011-05-07 20:11:52.429680,0
1,1304799112437500,-9.698196,-0.245077,-1.509888,0.213975,0.059683,-0.016308,33.449158,19.266598,15.818048,...,43.054999,-27.175334,-17.818824,JXL_SQ_trial1.xlsx,1,ADLs,1,SQ,2011-05-07 20:11:52.437500,0


In [8]:
# get the minimum time for each file/trial
times = df.groupby('FileName').min()['Time']

In [9]:
df['min_time'] = df.FileName.apply(lambda x: times[x])
df['time_seconds'] = (df.Time - df.min_time)/1000000

In [10]:
df.drop('min_time',axis=1,inplace=True)

<hr>

## Include column trial_num_by_subject_subtype

Some subjects have trial numbers as '1','2','4' or '2','4','5'. The code below restart the counting of the trials in 1 to 3. The original number of trials are stored in the column 'trial_num_original'

In [11]:
list_trial_nums = []
for type in list(df.trial_type.unique()):
    for subtype in list(df[df.trial_type == type].trial_subtype.unique()):
        for subject in list(df[(df.trial_type == type) & (df.trial_subtype == subtype)].subject.unique()):
#             print(subject)
            trials = list(df[(df.trial_type == type) & (df.trial_subtype == subtype) & (df.subject == subject)].trial_num_original.unique())
            trials = sorted(trials)
            trials = [int(x) for x in trials]
            
            trials_correct = [i+1 for i in range(len(trials))]
            trial_dict = dict()
            for i in range(3):
                trial_dict[trials[i]] = trials_correct[i]
                
            list_trial_nums.append([type, subtype, subject, trial_dict])

In [12]:
list_dfs = []
for index, item in enumerate(list_trial_nums):
    df_adjusts = df[(df.trial_type == item[0])&
                    (df.trial_subtype == item[1])&
                    (df.subject == item[2])                    
                    ]
    df_adjusts['trial_num'] = df_adjusts.trial_num_original.apply(lambda x: item[3][x])
    
    list_dfs.append(df_adjusts)

In [13]:
df_new = pd.concat(list_dfs, axis=0)

In [14]:
df_new.shape

(1190369, 73)

<hr>

# Metadata

To facilitate the data management, we'll store meta-information about the variables in a DataFrame. This will be helpful when we want to select specific variables for analysis, visualization, modeling, ...

Concretely we will store:

* Body Location: r.ankle, l.ankle, r.thigh, l.thigh, head, sternum, waist
* Axes: X, Y, Z
* Unit: m/s^2, rad/s, uT
* Measurements: aceleration, angular Velocity, magnetic field
* dtype: int, float, str

In [49]:
data = []
for f in df.columns:

    measure = ''
    # Defining the measure
    if 'cceleration' in f:
        measure = 'acceleration'
    elif 'ngular' in f:
        measure = 'angular velocity'
    elif 'agnetic' in f:
        measure = 'magnetic field'
         
    # Defining the body location
    body_location = ''
    if 'r.ankle' in f:
        body_location = 'r.ankle'
    elif 'l.ankle' in f:
        body_location = 'l.ankle'
    elif 'r.thigh' in f:
        body_location = 'r.thigh'
    elif 'l.thigh' in f:
        body_location = 'l.thigh'
    elif 'head' in f:
        body_location = 'head'
    elif 'sternum' in f:
        body_location = 'sternum'
    elif 'waist' in f:
        body_location = 'waist'

    axis = ''
    # Defining the Axes
    if ' X ' in f:
        axis = 'X'
    elif ' Y ' in f:
        axis = 'Y'
    elif ' Z ' in f:
        axis = 'Z'  
    elif ' resultant ' in f:
        axis = 'XYZ'
        
    # Defining the data type 
    dtype = df[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'body_location': body_location,
        'axis': axis,
        'measure': measure,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'body_location', 'axis', 'measure', 'dtype'])
meta.set_index('varname', inplace=True)

<hr>

## Calculating resultant measures

Consolidate the readings of the 3 axis using the equation below (example with waist acceleration)

df["waist resultant acceleration"] = $\sqrt{Acc_x^2 + Acc_y^2 + Acc_z^2}$

In [19]:
## TAKES LONG TIME TO RUN
for body_part in list(meta.body_location.unique())[1:]:
    print(body_part)
    for measure in list(meta.measure.unique())[1:]:
        col_name = str(body_part + ' resultant ' + measure)
        col_to_calculate = list(meta[(meta.body_location == body_part) & (meta.measure == measure)].index)
        df_new[col_name] = df_new.apply(lambda x: np.sqrt(x[col_to_calculate[0]]**2 + x[col_to_calculate[1]]**2 + x[col_to_calculate[2]]**2), axis=1)

r.ankle
l.ankle
r.thigh
l.thigh
head
sternum
waist


### Updating the metadata with the new columns

In [43]:
new_columns = list(filter(lambda k: 'resultant' in k, df_new.columns))
data = []
for f in new_columns:

    measure = ''
    # Defining the measure
    if 'acceleration' in f:
        measure = 'acceleration'
    elif 'angular velocity' in f:
        measure = 'angular velocity'
    elif 'magnetic field' in f:
        measure = 'magnetic field'
         
    # Defining the body location
    body_location = ''
    if 'r.ankle' in f:
        body_location = 'r.ankle'
    elif 'l.ankle' in f:
        body_location = 'l.ankle'
    elif 'r.thigh' in f:
        body_location = 'r.thigh'
    elif 'l.thigh' in f:
        body_location = 'l.thigh'
    elif 'head' in f:
        body_location = 'head'
    elif 'sternum' in f:
        body_location = 'sternum'
    elif 'waist' in f:
        body_location = 'waist'

    # Defining the Axes
    axis = 'XYZ'         
    
    # Defining the data type 
    dtype = df[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'body_location': body_location,
        'axis': axis,
        'measure': measure,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta_update = pd.DataFrame(data, columns=['varname', 'body_location', 'axis', 'measure', 'dtype'])
meta_update.set_index('varname', inplace=True)    

In [55]:
meta = pd.concat([meta,meta_update],axis=0)
# meta

,body_location,axis,measure,dtype
varname,,,,
Time,,,,int64
r.ankle Acceleration X (m/s^2),r.ankle,X,acceleration,float64
r.ankle Acceleration Y (m/s^2),r.ankle,Y,acceleration,float64
r.ankle Acceleration Z (m/s^2),r.ankle,Z,acceleration,float64
r.ankle Angular Velocity X (rad/s),r.ankle,X,angular velocity,float64
r.ankle Angular Velocity Y (rad/s),r.ankle,Y,angular velocity,float64
r.ankle Angular Velocity Z (rad/s),r.ankle,Z,angular velocity,float64
r.ankle Magnetic Field X (uT),r.ankle,X,magnetic field,float64
r.ankle Magnetic Field Y (uT),r.ankle,Y,magnetic field,float64


In [57]:
# remove rows to keep only those related with sensor measures
meta = meta[meta.body_location.str.len()>1]

## Reorganizing column names

In [20]:
# Reorganizing column names
df_new = df_new[['Time', 'time_datetime', 'time_seconds', 'subject', 'trial_type',
       'trial_num', 'trial_num_original', 'trial_subtype', 'r.ankle Acceleration X (m/s^2)',
       'r.ankle Acceleration Y (m/s^2)', 'r.ankle Acceleration Z (m/s^2)',
       'r.ankle Angular Velocity X (rad/s)',
       'r.ankle Angular Velocity Y (rad/s)',
       'r.ankle Angular Velocity Z (rad/s)', 'r.ankle Magnetic Field X (uT)',
       'r.ankle Magnetic Field Y (uT)', 'r.ankle Magnetic Field Z (uT)',
       'l.ankle Acceleration X (m/s^2)', 'l.ankle Acceleration Y (m/s^2)',
       'l.ankle Acceleration Z (m/s^2)', 'l.ankle Angular Velocity X (rad/s)',
       'l.ankle Angular Velocity Y (rad/s)',
       'l.ankle Angular Velocity Z (rad/s)', 'l.ankle Magnetic Field X (uT)',
       'l.ankle Magnetic Field Y (uT)', 'l.ankle Magnetic Field Z (uT)',
       'r.thigh Acceleration X (m/s^2)', 'r.thigh Acceleration Y (m/s^2)',
       'r.thigh Acceleration Z (m/s^2)', 'r.thigh Angular Velocity X (rad/s)',
       'r.thigh Angular Velocity Y (rad/s)',
       'r.thigh Angular Velocity Z (rad/s)', 'r.thigh Magnetic Field X (uT)',
       'r.thigh Magnetic Field Y (uT)', 'r.thigh Magnetic Field Z (uT)',
       'l.thigh Acceleration X (m/s^2)', 'l.thigh Acceleration Y (m/s^2)',
       'l.thigh Acceleration Z (m/s^2)', 'l.thigh Angular Velocity X (rad/s)',
       'l.thigh Angular Velocity Y (rad/s)',
       'l.thigh Angular Velocity Z (rad/s)', 'l.thigh Magnetic Field X (uT)',
       'l.thigh Magnetic Field Y (uT)', 'l.thigh Magnetic Field Z (uT)',
       'head Acceleration X (m/s^2)', 'head Acceleration Y (m/s^2)',
       'head Acceleration Z (m/s^2)', 'head Angular Velocity X (rad/s)',
       'head Angular Velocity Y (rad/s)', 'head Angular Velocity Z (rad/s)',
       'head Magnetic Field X (uT)', 'head Magnetic Field Y (uT)',
       'head Magnetic Field Z (uT)', 'sternum Acceleration X (m/s^2)',
       'sternum Acceleration Y (m/s^2)', 'sternum Acceleration Z (m/s^2)',
       'sternum Angular Velocity X (rad/s)',
       'sternum Angular Velocity Y (rad/s)',
       'sternum Angular Velocity Z (rad/s)', 'sternum Magnetic Field X (uT)',
       'sternum Magnetic Field Y (uT)', 'sternum Magnetic Field Z (uT)',
       'waist Acceleration X (m/s^2)', 'waist Acceleration Y (m/s^2)',
       'waist Acceleration Z (m/s^2)', 'waist Angular Velocity X (rad/s)',
       'waist Angular Velocity Y (rad/s)', 'waist Angular Velocity Z (rad/s)',
       'waist Magnetic Field X (uT)', 'waist Magnetic Field Y (uT)',
       'waist Magnetic Field Z (uT)','r.ankle resultant acceleration', 'r.ankle resultant angular velocity',
       'r.ankle resultant magnetic field', 'l.ankle resultant acceleration',
       'l.ankle resultant angular velocity',
       'l.ankle resultant magnetic field', 'r.thigh resultant acceleration',
       'r.thigh resultant angular velocity',
       'r.thigh resultant magnetic field', 'l.thigh resultant acceleration',
       'l.thigh resultant angular velocity',
       'l.thigh resultant magnetic field', 'head resultant acceleration',
       'head resultant angular velocity', 'head resultant magnetic field',
       'sternum resultant acceleration', 'sternum resultant angular velocity',
       'sternum resultant magnetic field', 'waist resultant acceleration',
       'waist resultant angular velocity', 'waist resultant magnetic field', 'FileName', 'target']]

In [21]:
# save the dataframe in a Pickle file
pickle.dump(df_new, open("../data/dataset_consolidated.p", "wb"))

In [58]:
# save the dataframe
pickle.dump(meta, open("../data/metadata.p", "wb"))